In [10]:
import random
import itertools
from typing import Iterator
import matplotlib.pyplot as plt
import time
import csv
import numpy as np

city = complex


def getX(A):
    return A.real


def getY(A):
    return A.imag


def distance(A, B):
    return abs(A - B)


def getCities(n, a=200, b=200):
    return {city(random.randrange(0, a), random.randrange(0, b)) for i in range(0, n)}


def tourLength(tour):
    return sum(distance(tour[i], tour[i-1]) for i in range(len(tour)))


def shortestTour(tours):
    return min(tours, key=tourLength)


def reverse_segment_if_better(tour, i, j):
    try:
        A, B, C, D = tour[i-1], tour[i], tour[j-1], tour[j % len(tour)]
        if distance(A, B) + distance(C, D) > distance(A, C) + distance(B, D):
            tour[i:j] = reversed(tour[i:j])
    except:
        pass


def lines(text): return text.strip().splitlines()


def Coordinate_map(lines, delimiter=' ', lat_col=1, long_col=2, lat_scale=69, long_scale=-48):
    return set(city(long_scale * float(row[long_col]),
                    lat_scale * float(row[lat_col]))
               for row in csv.reader(lines, delimiter=delimiter, skipinitialspace=True))


def read_cities(size):
    cities = []
    with open(f'test_data/cities_{size}.data', 'r') as handle:
        lines = handle.readlines()
        for line in lines:
            x, y = map(float, line.split())
            cities.append(city(x, y))
    return set(cities)


def allTours(cities):
    return itertools.permutations(cities)


def all_segments(N):
    return [(start, start + length)
            for length in range(N, 2-1, -1)
            for start in range(N - length + 1)]


def tspPlot(tour, subplot=True, iteration=1, title=""):
    x = [getX(i) for i in tour]
    y = [getY(i) for i in tour]

    x.append(x[0])
    y.append(y[0])

    if subplot:
        plt.subplot(1, 2, 1)

    plt.plot(x, y, marker='o', color='c', lw=1)
    plt.xlabel("X - axis")
    plt.ylabel("Y - axis")

    if len(tour) < 50:

        # zip joins x and y coordinates in pairs
        for a, b in zip(x, y):

            label = f"({a},{b})"

            plt.annotate(label,  # this is the text
                         (a, b),  # these are the coordinates to position the label
                         textcoords="offset points",  # how to position the text
                         xytext=(0, 5),  # distance from text to points (x,y)
                         fontsize=10)

    if subplot:
        axes = plt.gca()
        y_min, y_max = axes.get_ylim()
        x_min, x_max = axes.get_xlim()

        label = f"Distance: {round(tourLength(tour),2)}"
        generation = f"Generation: {iteration}"
        plt.text(x_min + abs(x_max / 25), y_min + y_max / 25, label)
        plt.text(x_min + abs(x_max / 25), y_min + (2 * y_max) / 25, generation)
        plt.title(f"TSP visualization")

    else:
        plt.title(title)

    # if iteration % 250 == 0:
    #     name = f"fig{iteration / 250}.png"
    #     plt.savefig(name)


def performancePlot(a):
    plt.subplot(1, 2, 2)
    x = range(0, len(a))
    y = a

    plt.xlabel("Number of Iteration")
    plt.ylabel("Best tour length")
    plt.title("GA Performance")

    plt.plot(x, y, color='c', lw=1)

In [11]:
USA_map = Coordinate_map(lines("""
[TCL]  33.23   87.62  Tuscaloosa,AL
[FLG]  35.13  111.67  Flagstaff,AZ
[PHX]  33.43  112.02  Phoenix,AZ
[PGA]  36.93  111.45  Page,AZ
[TUS]  32.12  110.93  Tucson,AZ
[LIT]  35.22   92.38  Little Rock,AR
[SFO]  37.62  122.38  San Francisco,CA
[LAX]  33.93  118.40  Los Angeles,CA
[SAC]  38.52  121.50  Sacramento,CA
[SAN]  32.73  117.17  San Diego,CA
[SBP]  35.23  120.65  San Luis Obi,CA
[EKA]  41.33  124.28  Eureka,CA
[DEN]  39.75  104.87  Denver,CO
[DCA]  38.85   77.04  Washington/Natl,DC
[MIA]  25.82   80.28  Miami Intl,FL
[TPA]  27.97   82.53  Tampa Intl,FL
[JAX]  30.50   81.70  Jacksonville,FL
[TLH]  30.38   84.37  Tallahassee,FL
[ATL]  33.65   84.42  Atlanta,GA
[BOI]  43.57  116.22  Boise,ID
[CHI]  41.90   87.65  Chicago,IL
[IND]  39.73   86.27  Indianapolis,IN
[DSM]  41.53   93.65  Des Moines,IA
[SUX]  42.40   96.38  Sioux City,IA
[ICT]  37.65   97.43  Wichita,KS
[LEX]  38.05   85.00  Lexington,KY
[NEW]  30.03   90.03  New Orleans,LA
[BOS]  42.37   71.03  Boston,MA
[PWM]  43.65   70.32  Portland,ME
[BGR]  44.80   68.82  Bangor,ME
[CAR]  46.87   68.02  Caribou Mun,ME
[DET]  42.42   83.02  Detroit,MI
[STC]  45.55   94.07  St Cloud,MN
[DLH]  46.83   92.18  Duluth,MN
[STL]  38.75   90.37  St Louis,MO
[JAN]  32.32   90.08  Jackson,MS
[BIL]  45.80  108.53  Billings,MT
[BTM]  45.95  112.50  Butte,MT
[RDU]  35.87   78.78  Raleigh-Durh,NC
[INT]  36.13   80.23  Winston-Salem,NC
[OMA]  41.30   95.90  Omaha/Eppley,NE
[LAS]  36.08  115.17  Las Vegas,NV
[RNO]  39.50  119.78  Reno,NV
[AWH]  41.33  116.25  Wildhorse,NV
[EWR]  40.70   74.17  Newark Intl,NJ
[SAF]  35.62  106.08  Santa Fe,NM
[NYC]  40.77   73.98  New York,NY
[BUF]  42.93   78.73  Buffalo,NY
[ALB]  42.75   73.80  Albany,NY
[FAR]  46.90   96.80  Fargo,ND
[BIS]  46.77  100.75  Bismarck,ND
[CVG]  39.05   84.67  Cincinnati,OH
[CLE]  41.42   81.87  Cleveland,OH
[OKC]  35.40   97.60  Oklahoma Cty,OK
[PDX]  45.60  122.60  Portland,OR
[MFR]  42.37  122.87  Medford,OR
[AGC]  40.35   79.93  Pittsburgh,PA
[PVD]  41.73   71.43  Providence,RI
[CHS]  32.90   80.03  Charleston,SC
[RAP]  44.05  103.07  Rapid City,SD
[FSD]  43.58   96.73  Sioux Falls,SD
[MEM]  35.05   90.00  Memphis Intl,TN
[TYS]  35.82   83.98  Knoxville,TN
[CRP]  27.77   97.50  Corpus Chrst,TX
[DRT]  29.37  100.92  Del Rio,TX
[IAH]  29.97   95.35  Houston,TX
[SAT]  29.53   98.47  San Antonio,TX
[LGU]  41.78  111.85  Logan,UT
[SLC]  40.78  111.97  Salt Lake Ct,UT
[SGU]  37.08  113.60  Saint George,UT
[CNY]  38.77  109.75  Moab,UT
[MPV]  44.20   72.57  Montpelier,VT
[RIC]  37.50   77.33  Richmond,VA
[BLI]  48.80  122.53  Bellingham,WA
[SEA]  47.45  122.30  Seattle,WA
[ALW]  46.10  118.28  Walla Walla,WA
[GRB]  44.48   88.13  Green Bay,WI
[MKE]  42.95   87.90  Milwaukee,WI
[CYS]  41.15  104.82  Cheyenne,WY
[SHR]  44.77  106.97  Sheridan,WY
"""))

In [29]:
def nearestNeighbor(cities, start=None):
    n = len(cities)
    citiesList = list(cities)
    if start is None:
        start = citiesList[0]
    shortesttour = [start]
    citiesList.remove(start)
    c = shortesttour[0]

    while len(shortesttour) != n:
        cost = 100000000
        best = None
        for ci in citiesList:
            if distance(c, ci) < cost:
                cost = distance(c, ci)
                best = ci
        shortesttour.append(best)
        citiesList.remove(best)
        c = best

    return shortesttour


def greedyCandidates(cities, greedy_seed):
    candidates = []
    for i in range(0, greedy_seed):
        startCity = random.choice(list(cities))
        candidates.append(nearestNeighbor(cities, startCity))

    return candidates


def createInitialPopulation(cities, num_candidates, greedy_seed):
    candidates = createCandidates(cities, num_candidates - greedy_seed)
    candidates.extend(greedyCandidates(cities, greedy_seed))
    return candidates


def orderCrossover(p1, p2):
    n = len(p1)
    a, b = random.randint(0, n - 1), random.randint(0, n - 1)

    if a > b:
        a, b = b, a

    c1 = p1[a:b+1]
    c2 = p2[a:b+1]

    c1 = list(set(c1).union(set(p2)))
    c2 = list(set(c2).union(set(p1)))

    return c1, c2


def partiallyMappedCrossover(p1, p2):
    n = len(p1)
    a, b = random.randint(0, n - 1), random.randint(0, n - 1)
    if a > b:
        a, b = b, a

    c1 = [None for i in range(0, n)]
    c2 = [None for i in range(0, n)]
    set1 = {}
    set2 = {}

    for i in range(a, b + 1):
        c1[i] = p1[i]
        set1[p1[i]] = p2[i]
        set2[p2[i]] = p1[i]
        c2[i] = p2[i]

    for i in range(0, a):
        if p1[i] not in c2:
            c2[i] = p1[i]

        if p2[i] not in c1:
            c1[i] = p2[i]

    for i in range(b + 1, n):
        if p1[i] not in c2:
            c2[i] = p1[i]

        if p2[i] not in c1:
            c1[i] = p2[i]

    for i in range(0, n):
        if c1[i] is None:
            x = set1[p2[i]]
            while x in c1:
                x = set1[x]
            c1[i] = x

        if c2[i] is None:
            x = set2[p1[i]]
            while x in c2:
                x = set2[x]
            c2[i] = x

    return c1, c2


def cycleCrossover(tours):
    pass


def tournamentSelection(tours, n, k):
    newTours = []
    for i in range(0, int(n / 2)):
        parentCandidates = sorted([random.choice(tours)
                                  for i in range(0, k)], key=tourLength)
        p1, p2 = parentCandidates[0], parentCandidates[1]
        c1, c2 = partiallyMappedCrossover(p1, p2)
        newTours.append(c1)
        newTours.append(c2)

    return newTours


def randomSelection(tours, n):
    newTours = []
    for i in range(0, int(n / 2)):
        p1, p2 = random.choice(tours), random.choice(tours)
        c1, c2 = partiallyMappedCrossover(p1, p2)
        newTours.append(c1)
        newTours.append(c2)

    return newTours


def rouletteWheelSelection(tours, n):

    pass


def rankSelection(tours, n):
    sorted(tours, key=tourLength)
    i = 0
    newTours = []
    while i < n:
        c1, c2 = partiallyMappedCrossover(tours[i], tours[i + 1])
        newTours.append(c1)
        newTours.append(c2)

        i += 2

    return newTours


def generateOffspring(tours, num_offsprings=180,  selection_type=3, tournament_k=4):
    newTours = []
    n = len(tours)
    if selection_type == 1:
        newTours = rankSelection(tours, n)

    elif selection_type == 2:
        newTours = randomSelection(tours, num_offsprings)

    elif selection_type == 3:
        newTours = tournamentSelection(tours, num_offsprings, tournament_k)

    newTours = sorted(newTours, key=tourLength)
    return newTours[: num_offsprings]


def createCandidates(citiesSet, num_candidates=200):
    cities = list(citiesSet)
    return [random.sample(cities, len(cities)) for i in range(0, num_candidates)]


def swapEdgeMutation(tour, i, j):
    reverse_segment_if_better(tour, i, j)


def swapMutation(tour, i, j):
    newTour = tour.copy()
    newTour[i], newTour[j] = newTour[j], newTour[i]
    if tourLength(newTour) < tourLength(tour):
        return newTour

    return tour


def scrambleMutation(tour, i, j):
    oldTour = tour.copy()
    newTour = oldTour[i:j+1]
    random.shuffle(newTour)
    oldTour[i:j+1] = newTour

    if tourLength(oldTour) < tourLength(tour):
        return oldTour
    else:
        return tour


def inversionMutation(tour, i, j):
    oldTour = tour.copy()
    newTour = oldTour[i:j+1]
    newTour.reverse()
    oldTour[i:j+1] = newTour

    if tourLength(oldTour) < tourLength(tour):
        return oldTour
    else:
        return tour


def generateRandomAB(n):
    a, b = random.randint(0, n - 1), random.randint(0, n - 1)
    a, b = min(a, b), max(a, b)
    return a, b


def mutate(offsprings, n, mutation_rate=0.05, mutation_type=1):
    j = -1
    for i in range(0, int(len(offsprings) * mutation_rate)):
        a, b = generateRandomAB(n)
        if mutation_type == 1:
            swapMutation(offsprings[i], a, b)
        if mutation_type == 2:
            swapEdgeMutation(offsprings[i], a, b)
        elif mutation_type == 3:
            offsprings[i] = scrambleMutation(offsprings[i], a, b)
        else:
            offsprings[i] = inversionMutation(offsprings[i], a, b)

        j -= 1


def getSelectionType():
    print("Select Selection Type: ")

    print("1. Rank Selection")
    print("2. Random Selection")
    print("3. Tournament Selection")

    choice = int(input())
    return choice


def getMutationType():
    print("Select Mutation Type: ")
    print("1. Swap Mutation")
    print("2. Swap Edge Mutation")
    print("3. Scramble Mutation")
    print("4. Inversion Mutation")

    choice = int(input())
    return choice

In [ ]:
import random
import itertools
import matplotlib.pyplot as plt
import time
import numpy as np

city = complex


def nearestNeighbor(cities, start=None):
    n = len(cities)
    citiesList = list(cities)
    if start is None:
        start = citiesList[0]
    shortesttour = [start]
    citiesList.remove(start)
    c = shortesttour[0]

    while len(shortesttour) != n:
        cost = 100000000
        best = None
        for ci in citiesList:
            if distance(c, ci) < cost:
                cost = distance(c, ci)
                best = ci
        shortesttour.append(best)
        citiesList.remove(best)
        c = best

    return shortesttour


def repeatedNearestNeighbor(cities):
    cost = 100000000
    tour = None
    performance = []
    plt.rcParams["figure.figsize"] = (12, 5)
    plt.tight_layout()

    for c in cities:
        temptour = nearestNeighbor(cities, c)
        tempcost = tourLength(temptour)
        performance.append(tempcost)
        if tempcost < cost:
            cost = tempcost
            tour = temptour
        plt.pause(0.05)
        plt.clf()
        performancePlot(performance)

        tspPlot(temptour)

    plt.show()

    return tour


def alter_tour(tour):
    original_length = tourLength(tour)
    for (start, end) in all_segments(len(tour)):
        reverse_segment_if_better(tour, start, end)

    if tourLength(tour) < original_length:
        return alter_tour(tour)
    return tour


def geneticAlgorithm(cities, iterations=2000, num_candidates=200, mutation_rate=0.05, num_elite=20, greedy_seed=10, selection_type=1, mutation_type=1):
    file = open("output.txt", "a")
    candidates = createInitialPopulation(
        cities, num_candidates, greedy_seed)
    bestSolution = shortestTour(candidates)
    bestCost = tourLength(bestSolution)
    sorted(candidates, key=tourLength)

    fitness = []
    plt.rcParams["figure.figsize"] = (15, 5)
    plt.tight_layout()

    n = 0

    while n <= iterations:
        elite = candidates[:num_elite]
        offsprings = generateOffspring(
            tours=candidates, num_offsprings=num_candidates - num_elite, selection_type=seletion_type)
        mutate(offsprings=offsprings, n=len(bestSolution),
               mutation_rate=mutation_rate, mutation_type=mutation_type)

        candidates = offsprings + elite

        tempBest = shortestTour(candidates)
        tempCost = tourLength(tempBest)
        fitness.append(tempCost)
        if tempCost < bestCost:
            plt.pause(0.05)
            bestSolution = tempBest
            bestCost = tempCost
            file.write(
                f"Generation: {n} Minimum Tour Length: {round(tempCost,4)}\n")

        plt.pause(0.05)
        plt.clf()
        performancePlot(fitness)

        tspPlot(tour=tempBest, subplot=True, iteration=n)

        n += 1

    plt.show()
    file.close()

    return bestSolution


if __name__ == "__main__":
    print("1. Generate Random Cities")
    print("2. USA Map")
    print("3. Run on Test Case")
    choice = int(input())

    if choice == 1:
        print("Enter the number of cities: ", end="")
        num_cities = int(input())

        cities = getCities(num_cities)

    elif choice == 2:
        cities = USA_map

    else:
        cities = read_cities(64)

    iterations = int(input("Number of Iterations: "))
    num_candidates = int(input("Population Size: "))
    elite = int(input("Number of Elite Parents: "))
    greedy_seed = int(input("Number of Greedy Seed: "))
    mutation_rate = float(input("Mutation Rate: "))
    seletion_type = getSelectionType()
    mutation_type = getMutationType()

    plt.rcParams["figure.figsize"] = (5, 5)
    tspPlot(list(cities), subplot=False,
            title="Initial Tour with tour length " + str(round(tourLength(list(cities)), 4)))
    plt.savefig('initial tour.png')

    plt.rcParams["figure.figsize"] = (15, 5)
    shortesttour = geneticAlgorithm(cities=cities, iterations=iterations, num_candidates=num_candidates,
                                    num_elite=elite, mutation_rate=mutation_rate, greedy_seed=greedy_seed, selection_type=seletion_type, mutation_type=mutation_type)

    print("Solution found with length: ", end="")
    print(tourLength(shortesttour))

    plt.rcParams["figure.figsize"] = (5, 5)
    tspPlot(shortesttour, subplot=False, title="Optimized Tour with tour length " +
            str(round(tourLength(shortesttour), 4)))
    plt.show()